<a href="https://colab.research.google.com/github/Nov05/DS-Unit-1-Sprint-5-Data-Storytelling-Blog-Post/blob/master/s%26p_banks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# created by nov05 on 2019-06-27

In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import stats

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import seaborn as sns

%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [5]:
# CAUTION: this dataset is 719 MB in size.

# this doesn't work for this large file (seems that it would work for file under 50 MB)
# url = "https://drive.google.com/uc?id=1giGHkApdv3ehRI7uw4LoLhGT6rN_uR0r"

# this works
# url = "https://data.consumerfinance.gov/api/views/s6ew-h6mp/rows.csv?accessType=DOWNLOAD"
# df = pd.read_csv(url)

# this works
path = "/content/drive/My Drive/data/cfpd_consumer_complaints/Consumer_Complaints.csv"
df = pd.read_csv(path)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# convert datetime
df['datereceived'] = pd.to_datetime(df['Date received'], 
                                    format="%m/%d/%Y")
df['datesent'] = pd.to_datetime(df['Date sent to company'],
                                format="%m/%d/%Y")
df[['datereceived', 'datesent']].sample(4)

,datereceived,datesent
1060358,2014-06-16,2014-06-16
358098,2018-01-24,2018-01-24
361060,2018-01-20,2018-01-20
295889,2018-04-12,2018-04-13


In [0]:
# consolidate changes in products - such a pain

def new_prd(row):
  prd, sub = row[0], row[1]
  new_prd = ""
  
  # Debt collection (no change to product name)
  if prd in [prds[7]]:
    new_prd = prds[7]
  # Credit reporting, credit repair services, or other 
  # personal consumer reports (previously “Credit reporting”
  # and sub-products of “Other financial service”)
  elif prd in [prds[5], prds[6]]:
    new_prd = prds[6]
  # Mortgage (no change to product name)
  elif prd in [prds[10]]:
    new_prd = prds[10]
  # Credit card or prepaid card (previously “Credit card” 
  # and “Prepaid card”)  
  elif prd in [prds[3], prds[14], prds[4]]:
    new_prd = prds[4]
  # Checking or savings account (previously Bank account 
  # or service)  
  elif prd in [prds[0], prds[1]]:
    new_prd = prds[1]
  # Vehicle loan or lease (previously sub-product of 
  # Consumer loan)  
  elif prd==prds[16] or (prd==prds[2] and sub==subs[8]):
    new_prd = prds[16]
  # Student loan (no change to product name)  
  elif prd in [prds[15]]:
    new_prd = prds[15]
  # Payday loan, title loan, or personal loan (previously 
  # “Consumer loan” and “Payday loan”)
  elif prd in [prds[13], prds[12]] or (prd==prds[2] 
                                       and sub!=subs[8]):
    new_prd = prds[13]
  # Money transfer, virtual currency, or money service 
  # (previously “Money transfers,” “Other financial 
  # service,” and “Prepaid card”)  
  elif prd in [prds[8], prds[17], prds[9], prds[11], 
               prds[14]]:
    new_prd = prds[8]
  
  return new_prd

In [8]:
df_prd = (df.fillna('-')
   .groupby(by=['Product', 'Sub-product'], as_index=False)
   .agg({'Complaint ID': pd.Series.nunique})
   .sort_values(by=['Product', 'Complaint ID'], 
                ascending=(True, False))
)
# print(df_prd.shape) # (99, 3)

prds = df_prd['Product'].unique()
subs = df_prd['Sub-product'].unique()

# consolidate products for the whole dataset
df['new'] = df.iloc[:,1:3].apply(new_prd, axis=1)
# verify consolidation - result data frame should be empty
df[df['new']=='']

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,datereceived,datesent,new


S&P Banks Select Industry Index  
Ticker:  SPSIBK  
https://us.spindices.com/indices/equity/sp-banks-select-industry-index  

**S&P Select Industry Index**                      
S&P Banks Select Industry Index    
**Theoretical Portfolio Value**    
$2,000,000,000  
https://us.spindices.com/documents/methodologies/methodology-sp-select-industry-indices.pdf?force_download=true  

**Large Commercial Banks**  
https://www.federalreserve.gov/releases/lbr/current/  
https://www.federalreserve.gov/releases/lbr/  

**CSV file on Gist**  
https://gist.githubusercontent.com/Nov05/91a6ba49ede748ad32998e0c31a2fa35/raw/c92e0b758087b16723f11832432a033efec89bf5/9.csv  

In [34]:
# url = "https://www.federalreserve.gov/releases/lbr/current/lrg_bnk_lst.txt"

# ASCII file - better find a better data source
# yeah copy the table data then convert it from tab separated file to CSV
# I love myself. Haha.

url = "https://gist.githubusercontent.com/Nov05/91a6ba49ede748ad32998e0c31a2fa35/raw/c92e0b758087b16723f11832432a033efec89bf5/9.csv"
df_banks = pd.read_csv(url)
print(df_banks.shape)
df_banks.head(2)

(1817, 13)


,Bank Name / Holding Co Name,Nat'l Rank,Bank ID,Bank Location,Charter,Consol Assets (Mil $),Domestic Assets (Mil $),Pct Domestic Assets,Pct Cumulative Assets,Domestic Branches,Foreign Branches,IBF,Pct Foreign Owned
0,JPMORGAN CHASE BK NA/JPMORGAN CHASE & CO,1,852218,"COLUMBUS, OH",NAT,"2,218,960","1,659,733",75,14,"5,041",33,Y,0.00
1,BANK OF AMER NA/BANK OF AMER CORP,2,480228,"CHARLOTTE, NC",NAT,"1,782,639","1,690,841",95,25,"4,338",27,Y,0.00


In [44]:
# check NAs
print(df_banks.isna().sum().sum(),
df['Company'].isna().sum())

0 0


In [39]:
# convert numeric field
(df_banks['Consol Assets (Mil $)']
 .replace(",", "", regex=True, inplace=True)
)
df_banks['Consol Assets (Mil $)'].head(2)

0    2218960
1    1782639
Name: Consol Assets (Mil $), dtype: object

In [40]:
type(df_banks.iloc[0,6])

str

In [55]:
# check unique bank names and test text processing
df_names = pd.DataFrame(df['Company'].unique())
df_names[0] = df_names[0].str.upper()
df_names.replace('\.', '', regex=True, inplace=True)
print(df_names.shape)
df_names.head()

(5321, 1)


,0
0,NATIONWIDE RECOVERY GROUP LLC
1,"ONEMAIN FINANCIAL HOLDINGS, LLC"
2,"HCFS HEALTH CARE FINANCIAL SERVICES, INC"
3,NAVY FEDERAL CREDIT UNION
4,FIRST CREDIT FINANCE


In [64]:
# process texts
df['bankname'] = (df['Company']
                  .str
                  .upper()
                  .replace('\.', '', regex=True))
df['bankname'].head()

0               NATIONWIDE RECOVERY GROUP LLC
1             ONEMAIN FINANCIAL HOLDINGS, LLC
2    HCFS HEALTH CARE FINANCIAL SERVICES, INC
3                   NAVY FEDERAL CREDIT UNION
4                        FIRST CREDIT FINANCE
Name: bankname, dtype: object

In [67]:
# join two tables by name, god bless you
# (should be joined by bank id, but there are no bank ids
#  in the complaint dataset...)

df_all = pd.merge(df, df_banks, how='inner',
                  left_on = 'bankname',
                  right_on = 'Bank Name / Holding Co Name'
                 )
print(df_all.shape)
df_all[['bankname', 'Consol Assets (Mil $)']].head()

(0, 35)


,bankname,Consol Assets (Mil $)


In [79]:
# test contains
df_names[df_names[0].str.contains('BANK OF AMERICA')]

,0
123,"BANK OF AMERICA, NATIONAL ASSOCIATION"


In [80]:
df_names[df_names[0].str.contains('JPMORGAN')]

,0
49,JPMORGAN CHASE & CO


In [121]:
# ok. match records manually for a few banks...
# s[s.str.contains('|'.join(searchfor))]
top20 = ['JPMORGAN',  
         'BANK OF AMER', 
         'WELLS FARGO',
         'CITIBANK',
         'US BANC',
         'PNC BANK',
         'CAPITAL ONE',
         'TD BANK',
         'NY MELLON',
         'STATE STREET',
         'BB&T',
         'SUNTRUST',
         'GOLDMAN SACHS',
         'HSBC',
         'ALLY F', 
         'MORGAN STANLEY',
         'FIFTH THIRD',
         'KEYCORP',
         'CHASE', # not found, it is probably with JPMORGAN
         'NORTHERN T',
         'MUFG'
        ]
df_matches = df_names[df_names[0].str.contains('|'.join(top20))]
print(len(top20))
print(df_matches.shape)
df_matches

21
(20, 1)


,0
8,ALLY FINANCIAL INC
29,US BANCORP
35,HSBC NORTH AMERICA HOLDINGS INC
49,JPMORGAN CHASE & CO
50,"CITIBANK, NA"
78,WELLS FARGO & COMPANY
123,"BANK OF AMERICA, NATIONAL ASSOCIATION"
147,BB&T CORPORATION
155,FIFTH THIRD FINANCIAL CORPORATION
182,CAPITAL ONE FINANCIAL CORPORATION


In [119]:
# 1. JPMORGAN CHASE BK NA/JPMORGAN CHASE & CO
# 20. CHASE BK USA NA/JPMORGAN CHASE & CO
df_names[df_names[0].str.contains('CHASE')]

,0
49,JPMORGAN CHASE & CO


In [122]:
df_all = pd.merge(df_matches, df, how='inner',
                  left_on=0, right_on='bankname'
                 )
print(df_all.shape)

(379862, 23)


# Finally it worked. Such a pain. Hate joining data on texts.

In [165]:
df_tops = (df_all
 .groupby(by='bankname', as_index=False)
 .agg({'Complaint ID':pd.Series.nunique})
 .sort_values(by='Complaint ID',
              ascending=False)
)
df_tops.index=[x for x in np.arange(20)]
print(df_tops.shape)
df_tops

(20, 2)


,bankname,Complaint ID
0,"BANK OF AMERICA, NATIONAL ASSOCIATION",83053
1,WELLS FARGO & COMPANY,71931
2,JPMORGAN CHASE & CO,61283
3,"CITIBANK, NA",49887
4,CAPITAL ONE FINANCIAL CORPORATION,35406
5,US BANCORP,17402
6,PNC BANK NA,11990
7,TD BANK US HOLDING COMPANY,9922
8,HSBC NORTH AMERICA HOLDINGS INC,8840
9,"SUNTRUST BANKS, INC",8800


In [143]:
df_banks.iloc[:21,0] 
# [1,2,0,3,6,4,5,7,13,11,14,16,10,17,20,15,12,8,19,9]

0         JPMORGAN CHASE BK NA/JPMORGAN CHASE & CO
1                BANK OF AMER NA/BANK OF AMER CORP
2               WELLS FARGO BK NA/WELLS FARGO & CO
3                            CITIBANK NA/CITIGROUP
4                                 U S BK NA/U S BC
5                     PNC BK NA/PNC FNCL SVC GROUP
6                    CAPITAL ONE NA/CAPITAL ONE FC
7                    T D BK NA/TD GRP US HOLDS LLC
8         BANK OF NY MELLON/BANK OF NY MELLON CORP
9              STATE STREET B&TC/STATE STREET CORP
10                         BRANCH BKG&TC/BB&T CORP
11                         SUNTRUST BK/SUNTRUST BK
12    GOLDMAN SACHS BK USA/GOLDMAN SACHS GROUP THE
13                HSBC BK USA NA/HSBC N AMER HOLDS
14                               ALLY BK/ALLY FNCL
15             MORGAN STANLEY BK NA/MORGAN STANLEY
16                   FIFTH THIRD BK/FIFTH THIRD BC
17                              KEYBANK NA/KEYCORP
18             CHASE BK USA NA/JPMORGAN CHASE & CO
19                    NORTHERN 

In [145]:
seq = [1,2,0,3,6,4,5,7,13,11,14,16,10,17,20,15,12,8,19,9]
len(seq)

20

In [155]:
df_topbanks = pd.DataFrame([df_banks.iloc[:21,[0,1,2,5,6]].values[s] for s in seq])
df_topbanks.head(3)

,0,1,2,3,4
0,BANK OF AMER NA/BANK OF AMER CORP,2,480228,1782639,"1,690,841"
1,WELLS FARGO BK NA/WELLS FARGO & CO,3,451965,1689351,"1,635,096"
2,JPMORGAN CHASE BK NA/JPMORGAN CHASE & CO,1,852218,2218960,"1,659,733"


In [166]:
pd.concat([df_tops, df_topbanks], axis=1)

,bankname,Complaint ID,0,1,2,3,4
0,"BANK OF AMERICA, NATIONAL ASSOCIATION",83053,BANK OF AMER NA/BANK OF AMER CORP,2,480228,1782639,"1,690,841"
1,WELLS FARGO & COMPANY,71931,WELLS FARGO BK NA/WELLS FARGO & CO,3,451965,1689351,"1,635,096"
2,JPMORGAN CHASE & CO,61283,JPMORGAN CHASE BK NA/JPMORGAN CHASE & CO,1,852218,2218960,"1,659,733"
3,"CITIBANK, NA",49887,CITIBANK NA/CITIGROUP,4,476810,1406717,"826,848"
4,CAPITAL ONE FINANCIAL CORPORATION,35406,CAPITAL ONE NA/CAPITAL ONE FC,7,112837,304658,"304,654"
5,US BANCORP,17402,U S BK NA/U S BC,5,504713,459477,"449,994"
6,PNC BANK NA,11990,PNC BK NA/PNC FNCL SVC GROUP,6,817824,370501,"366,312"
7,TD BANK US HOLDING COMPANY,9922,T D BK NA/TD GRP US HOLDS LLC,8,497404,302669,"302,669"
8,HSBC NORTH AMERICA HOLDINGS INC,8840,HSBC BK USA NA/HSBC N AMER HOLDS,14,413208,169812,"168,096"
9,"SUNTRUST BANKS, INC",8800,SUNTRUST BK/SUNTRUST BK,12,675332,209720,"209,720"
